# How magnetic waves are used to construct non-invasive internal anatomic and physiologic images 


## Let's start with how the way we hear sounds is similar to how we image with MRI:

When plucking a string, one can hear a note. That note comes from oscillation at a particular frequency. Notice one note has one frequency, while two notes will have two frequencies, and three notes will have three frequencies. This can be shown with the guitar! 

Imagine how useful it would be if instead of seeing just the sum of those sinusoids, instead we could get the frequency, amplitude, and phase of the sinusoids that are making up that note!

This is called a "fourier transform". This means that every single signal can be decomposed into the sum of sinusoids of varying frequencies. In the fourier domain, it is represented by a delta function at the frequency of the wave, with the amplitude and phase saved as the height of the delta function and the complex portion of the signal. 

In [100]:
import numpy as np
import matplotlib
import matplotlib.pyplot as pl
import pickle
import pydicom as dicom
import matplotlib.image as mpimg

In [4]:
%matplotlib

Using matplotlib backend: MacOSX


# Here's an example with a sound wave of 5Hz and a sound wave of 7Hz: 

Starting with one sinusoid, we can see the fourier transform of it in the fourier domain and how it shifts location as the frequency increases! 

Then, we can see how the combination of the two can be shown as the sum of two sinusoids in fourier space!

In [98]:
x = np.linspace(0, 1, 1000) #from 0 to 1, sampling rate of 1/1000
wave1 = np.sin(2*np.pi*5*x) #frequency of 5
ft_sin1 = np.fft.fft(wave1)
ft_sin1 = np.fft.fftshift(ft_sin1)
ft_freq = np.linspace(-500,500,1000) #kmax = 1/2Deltax, Delta kx = 1/FOVx

# Set up subplots
fig, ((ax1, ax2),(ax3,ax4),(ax5,ax6)) = pl.subplots(3,2)

# plot wave 1 with 5 cycles per second
ax1.plot(x,wave1)
ax1.set_title('5 Hz')
ax2.plot(ft_freq,abs(ft_sin))
ax2.set_title('FT of 5 Hz')
ax2.set_xlim([-10,10])

# plot wave 2 with 7 cycles per second
wave2 = np.sin(2*np.pi*7*x)
ft_sin2 = np.fft.fft(wave2)
ft_sin2 = np.fft.fftshift(ft_sin2)
ax3.plot(x,wave2,color = 'red')
ax3.set_title('7 Hz')
ax4.plot(ft_freq,abs(ft_sin2),color = 'red')
ax4.set_title('FT of 7 Hz')
ax4.set_xlim([-10,10])

# plot sum of the two
wavesum = wave1+wave2
ft_sinsum = np.fft.fft(wavesum)
ft_sinsum = np.fft.fftshift(ft_sinsum)
ax5.plot(x,wave1+wave2,color = 'purple')
ax5.set_title('5 Hz + 7 Hz')
ax6.plot(ft_freq,abs(ft_sinsum),color = 'purple')
ax6.set_title('FT of 7 Hz + 5 Hz')
ax6.set_xlim([-10,10])


fig.tight_layout()
pl.savefig('data/Fig1.png',dpi = 100)

with open('data/Fig1_1dWaves.pickle','wb') as handle: 
    pickle.dump([wave1,ft_sin1,wave2,ft_sin2,wavesum,ft_sinsum],handle)

In [104]:
img = mpimg.imread('data/Fig1.png')
pl.imshow(img)
pl.show()

# Now how can we see this in an image? 

Let's make those waves two dimensional! They're planar waves now, 2D! 

Of course, this means they will be two frequency space? 

That doesn't make any sense intuitively, does it... so let's take a look!

Here you see that the 2D planar wave of a single frequency is again actually just two delta functions in a 2D plane! 

So what does their location in the 2D space mean? 

In [105]:
fig, ((ax1, ax2),(ax3,ax4)) = pl.subplots(2,2)

x = np.arange(0, 101, 1)
X, Y = np.meshgrid(x, x)
img = np.sin(2 * np.pi * X /20) #20 is the wavelength (how many pixels in one lil wave)
ax1.imshow(img,cmap = 'gray')
ax1.set_title('2d planar wave')
ax2.plot(img[51])
ax2.set_title('Cross section of 2d planar wave')

#so this has wavelength of 30
# pixel size of 3

ft = np.fft.fft2(img)
kspace = np.fft.fftshift(ft) #moves zero frequency component to center
ax3.imshow(np.abs(kspace),cmap = 'gray')
ax3.set_title('Fourier Transform of 2d planar wave')
ax4.plot(np.abs(kspace)[50])
ax4.set_title('Cross section of Fourier Transform')
fig.tight_layout()
pl.show()


pl.savefig('data/Fig2.png',dpi = 100)



# 2D frequencies?

Here we can see that the location of the delta waves in the 2D space is the direction of the wave! 

So as 1D wave doesn't have any direction, it only has frequency. (this can be hard to understand, but if you think about it, even for a 1D wave to travel in a direction, that direction has to be a direction in at least 2D space!) 

But a 2D wave has both a frequency AND a direction in which it is going! This means that the direction of the frequency is also encoded in its location in the 2D frequency space! 

As we rotate the planar waves to have them propagating in different directions, we can see the delta functions also rotating in k-space!

In [107]:
fig, (ax1, ax2) = pl.subplots(1,2)
angles = np.linspace(0,np.pi,15)
ax1.set_title('Planar Wave')
ax2.set_title('FT of Planar Wave')
for tilt in angles:
    #phi being the tilt angle
    img = np.sin(2*np.pi*(X*np.cos(tilt) + Y*np.sin(tilt)) / 20)
    ax1.imshow(img,cmap = 'gray')
    ft = np.fft.fft2(img)
    kspace = np.fft.fftshift(ft) #moves zero frequency component to center
    ax2.imshow(np.abs(kspace),cmap = 'gray')

    pl.pause(.5)
    
    

In [14]:
fig, (ax1, ax2, ax3) = pl.subplots(1,3)
phases = np.linspace(0,np.pi,10)
ax1.set_title('Phase shifting\nPlanar Wave')
ax2.set_title('FT of Planar Wave')
ax3.set_title('imaginary component\nof iFT')
for phi in phases:
    #phi being the tilt angle
    img = np.sin(2*np.pi*(X*np.cos(np.pi/4) + Y*np.sin(np.pi/4)) / 20 + phi)
    ax1.imshow(img,cmap = 'gray')
    ft = np.fft.fft2(img)
    kspace = np.fft.fftshift(ft) #moves zero frequency component to center
    ax2.imshow(np.abs(kspace),cmap = 'gray')
    ax3.imshow(np.imag(kspace),cmap = 'gray')
    
    
    pl.colorbar
    pl.pause(.5)

In [33]:
name = 'Coronal' #Sagittal, Axial
ds = dicom.dcmread('data/' + name + '.dcm')
img = ds.pixel_array

fov_img = 256

with open('data/'+name + '_superposition.pickle','rb') as handle1:
    stack_image = pickle.load(handle1)
with open('data/'+ name +'_superposition_masks.pickle','rb') as handle2:
    stack_waves = pickle.load(handle2)
with open('data/'+ name + '_superposition_kspace.pickle','rb') as handle3:
    stack_kspace = pickle.load(handle3)
    
fig, (ax1, ax2, ax3,ax4) = pl.subplots(1,4)
ax4.imshow(img,cmap = 'gray')
ax4.set_title('Final Image')
for i in np.arange(0,np.shape(stack_image)[2]):
    ax1.imshow(stack_kspace[:,:,i],vmin = 0,vmax = 6000,extent = [-.5, .5, -.5, .5],cmap = 'gray')
    ax1.set_title('k-space')
    ax2.imshow(stack_waves[:,:,i],cmap = 'gray')
    ax2.set_title('planar\nwaves')
    ax3.imshow(stack_image[:,:,i],cmap = 'gray')
    ax3.set_title('Image\nForming')
    pl.pause(.1)